# What happens:
File is created and email is prepared

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/FX Split"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Select Data

In [4]:
%%time
d1 = pd.read_sql_query("""select reporting_date,  
booking_company,
calc_currency, 
c.exch_rate,
sum(rwa)/1000000 rwa_eur,  
sum(rwa)/1000000/c.exch_rate rwa_lcy, 
sum(ead_pre_ccf)/1000000 ead_pre_ccf  
from 
( 
select reporting_date, booking_company, currency as calc_currency, rwa, ead_pre_ccf from rzb_cdr 
where (contract_type not like 'PA0401' or contract_type is NULL) 
and (bs_type not in ('VIR', 'TRA') or bs_type is NULL) 
and (securitized not like 'T' or securitized is NULL) 
) r join currency c on (r.calc_currency=c.ccy_code) 
group by reporting_date, booking_company, calc_currency, exch_rate """, connection_RAY)

d2 = pd.read_sql_query("""select partition_key, ccy_code, ccy_desc, exch_rate from currency """, connection_RAY)

d3 = pd.read_sql_query("""select distinct c.currency, c. exchange_rate, 
sum(t.cva_rwa) cva_rwa_EUR_total, 
sum(t.cva_rwa)/c.exchange_rate cva_rwa_LCY_total, 
sum(case when dealbook like '%_T' then  t.cva_rwa else 0 End) as RWA_trading_book_EUR, 
sum(case when dealbook like '%_B' then t.cva_rwa else 0 End) as RWA_banking_book_EUR 
from t_cva t join (select distinct exposure_attribute_5, currency, exchange_rate from T_CDR) c on (t.exposure_attribute_5 = c.exposure_attribute_5) 
group by c.currency, c.exchange_rate 
order by currency """, connection_RAY)

Wall time: 8.56 s


### Save selections in excel file

In [5]:
#get last month
dt = datetime.datetime.today()
month_name = calendar.month_name[dt.month-1].upper()
year  = dt.year

# determining the name of the file
file_name = rd + '_FX_SPLIT_' + month_name + '_' + str(year) + '.xlsx'

writer = pd.ExcelWriter(file_name)
# saving the excel
d1.to_excel(writer, index=False, sheet_name = rd + '_Group Cons Data ')
d2.to_excel(writer, index=False, sheet_name = rd + '_Fx_rates_current ')
d3.to_excel(writer, index=False, sheet_name = rd + '_fx_cva_rwa  ')


writer.save()
print('DataFrame is written to Excel File successfully.')


DataFrame is written to Excel File successfully.


# You have another file to prepare
### Link: https://wiki.rbinternational.corp/confluence/display/OCR/FX-Split+-+upload+file+in+shared+folder


In [6]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'stefanie.gruen@rbinternational.com'
mail.Cc = 'aleksandr.mikayelyan@rbinternational.com; markus.hochleitner@rbinternational.com; alexandru-paul.gliga@rbinternational.com'
mail.Subject = rd + '_FX_SPLIT_' + month_name + '_' + str(year)
mail.Body = '''Dear Stefanie,

Please find attached the data as of ''' + month_name + ''' ''' + str(year) +'''. The file is also available in the RBI Capital Steering Sharepoint.

Thanks in advance and kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
# To attach a file to the email (optional):
attachment  = directory + "/" + file_name
mail.Attachments.Add(attachment)
mail.Display()